In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

In [2]:
heroinUsers = pd.read_excel('./tables/drugOffenses/heroinUsers.xlsx', 
                            usecols=[i for i in range(15)], nrows=30) # for a clean dataframe
heroinSuppliers = pd.read_excel('./tables/drugOffenses/heroinSuppliers.xlsx', 
                            usecols=[i for i in range(15)], nrows=30)

In [3]:
target_country = 'Austria' # target country for model

In [4]:
# making the dataframe for target_country
df = heroinSuppliers[heroinSuppliers['Country'] == target_country].T[1:]
df.columns = ["Suppliers " + target_country]

for country in list(heroinSuppliers['Country']):
    if country != target_country:
        joiner = heroinSuppliers[heroinSuppliers['Country'] == country].T[1:]
        joiner.columns = ["Suppliers " + country]
        df = df.join(joiner)

# users in target country
joiner = heroinUsers[heroinUsers['Country'] == target_country].T[1:]
joiner.columns = ["Users " + target_country]
df = df.join(joiner)

In [5]:
# handeling NAN values

for country in df.columns:
    if df[country].isnull().sum() >= len(df[country]/2):
        print("in " + country + " ima poveke od half")
        df = df.drop(country, axis = 1) # if more than half are nan
    else:
        df[country] = df[country].fillna(df[country].mean())

in Suppliers Denmark ima poveke od half
in Suppliers Estonia ima poveke od half
in Suppliers Finland ima poveke od half
in Suppliers Ireland ima poveke od half
in Suppliers Netherlands ima poveke od half
in Suppliers Norway ima poveke od half
in Suppliers Romania ima poveke od half
in Suppliers Sweden ima poveke od half


In [6]:
X = df.loc[:, df.columns != "Users " + target_country].values
y = df["Users " + target_country].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [9]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

LinearRegression()

In [10]:
y_pred = regressor.predict(X_test)

In [11]:
dfpred = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
dfpred

,Actual,Predicted
0,2453.0,2845.839353
1,1875.0,1684.878392
2,975.0,724.635468
3,2676.0,2625.671293
4,1018.0,1115.975494


In [13]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 196.3259389275675
Mean Squared Error: 53056.71173067564
Root Mean Squared Error: 230.3404257412833


## simple regression on Austria based on years and suppliers in Austria

In [14]:
target_country = 'Austria'
df_simple = heroinSuppliers[heroinSuppliers['Country'] == target_country].T[1:]
df_simple.columns = ["Suppliers " + target_country]

joiner = heroinUsers[heroinUsers['Country'] == target_country].T[1:]
joiner.columns = ["Users " + target_country]
df_simple = df_simple.join(joiner)
df_simple = df_simple.drop(2004)
df_simple['Years'] = [i for i in range(2005, 2018)][::-1]

In [15]:
reg_simple = linear_model.LinearRegression()
reg_simple.fit(df_simple[['Years','Suppliers Austria']], df_simple["Users Austria"])

LinearRegression()

In [16]:
user_list=[]

suppliers = df_simple['Suppliers Austria'].mean()
for year in range(2018, 2025):
    users = reg_simple.predict([[year, suppliers]])[0]
    print("In the year " + str(year) + " the model has predicted " + 
          str(users) + " users in Austria on " + str(suppliers) + " suppliers")
    
    user_list.append(users)    #on mean suppliers
print("====================")
suppliers = 700
for year in range(2018, 2025):
    users = reg_simple.predict([[year, suppliers]])[0]
    print("In the year " + str(year) + " the model has predicted " + 
          str(users) + " users in Austria on " + str(suppliers) + " suppliers")

In the year 2018 the model has predicted 1020.3006127455155 users in Austria on 586.5384615384615 suppliers
In the year 2019 the model has predicted 880.9919090718031 users in Austria on 586.5384615384615 suppliers
In the year 2020 the model has predicted 741.6832053980906 users in Austria on 586.5384615384615 suppliers
In the year 2021 the model has predicted 602.3745017243782 users in Austria on 586.5384615384615 suppliers
In the year 2022 the model has predicted 463.06579805066576 users in Austria on 586.5384615384615 suppliers
In the year 2023 the model has predicted 323.7570943769533 users in Austria on 586.5384615384615 suppliers
In the year 2024 the model has predicted 184.44839070324088 users in Austria on 586.5384615384615 suppliers
In the year 2018 the model has predicted 1379.8000689211185 users in Austria on 700 suppliers
In the year 2019 the model has predicted 1240.491365247406 users in Austria on 700 suppliers
In the year 2020 the model has predicted 1101.1826615736936 u

In [17]:
years = [i for i in range (2018,2025)]
plt.figure(figsize=(15,7))
plt.plot(years, user_list)